<a href="https://colab.research.google.com/github/prasanth-33460/matrice/blob/main/matrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorflow-addons
!pip install --upgrade tensorflow-addons

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade tensorflow

In [3]:
import os
import json
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import numpy as np

In [4]:
tf.__version__

'2.16.1'

In [5]:
appliance_annotations_path = "/content/drive/MyDrive/hello/appliance/annotations"
food_annotations_path = "/content/drive/MyDrive/hello/food/annotations"
appliance_images_path = "/content/drive/MyDrive/hello/appliance/images"
food_images_path = "/content/drive/MyDrive/hello/food/images"
train_subdir = "train"
test_subdir = "test"
val_subdir = "val"

In [6]:
print("Food TFRecord files:", os.listdir(food_annotations_path))
print("Appliance TFRecord files:", os.listdir(appliance_annotations_path))
print("Food train images:", os.listdir(os.path.join(food_images_path, train_subdir)))
print("Appliance train images:", os.listdir(os.path.join(appliance_images_path, train_subdir)))

Food TFRecord files: ['instances_val2017.json', 'instances_train2017.json', 'instances_test2017.json']
Appliance TFRecord files: ['instances_train2017.json', 'instances_val2017.json', 'instances_test2017.json']
Food train images: ['100765.jpg', '100134.jpg', '100270.jpg', '100564.jpg', '100331.jpg', '100377.jpg', '100440.jpg', '100632.jpg', '100394.jpg', '100246.jpg', '100376.jpg', '100795.jpg', '100399.jpg', '100393.jpg', '100336.jpg', '100333.jpg', '100778.jpg', '100570.jpg', '100142.jpg', '100082.jpg', '100623.jpg', '100079.jpg', '100758.jpg', '100406.jpg', '100379.jpg', '100204.jpg', '100006.jpg', '100527.jpg', '100125.jpg', '100344.jpg', '100665.jpg', '100743.jpg', '100165.jpg', '100627.jpg', '100329.jpg', '100042.jpg', '100297.jpg', '100521.jpg', '100166.jpg', '100085.jpg', '100467.jpg', '100722.jpg', '100290.jpg', '100542.jpg', '100089.jpg', '100233.jpg', '100465.jpg', '100129.jpg', '100797.jpg', '100455.jpg', '100170.jpg', '100671.jpg', '100684.jpg', '100589.jpg', '100025.jpg',

In [7]:
food_annotation_paths = [os.path.join(food_annotations_path, file) for file in os.listdir(food_annotations_path) if file.endswith('.json')]
appliance_annotation_paths = [os.path.join(appliance_annotations_path, file) for file in os.listdir(appliance_annotations_path) if file.endswith('.json')]

In [8]:
def load_annotations(annotation_paths):
    annotations = []
    for path in annotation_paths:
        with open(path, 'r') as f:
            annotation = json.load(f)
            annotations.append(annotation)
    return annotations


In [9]:
appliance_annotations = load_annotations(appliance_annotation_paths)
food_annotations = load_annotations(food_annotation_paths)

In [1]:
def parse_annotation(annotation, image_dir):
    image_id = annotation['annotations'][0]['image_id']
    image_path = os.path.join(image_dir, f"{image_id}.jpg")

    boxes = []
    labels = []

    for ann in annotation['annotations']:
        bbox = ann['bbox']
        boxes.append([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
        labels.append(ann['category_id'])

    if len(boxes) != len(labels):
        raise ValueError("Number of boxes and labels do not match.")

    return image_path, boxes, labels

In [44]:
def create_tf_dataset(annotations, image_dir):
    image_paths = []
    boxes_list = []
    labels_list = []

    for annotation in annotations:
        image_path, boxes, labels = parse_annotation(annotation, image_dir)
        image_paths.append(image_path)
        boxes_list.append(boxes)
        labels_list.append(labels)

    max_length = max([len(l) for l in labels_list])

    labels_list = [l + [0] * (max_length - len(l)) for l in labels_list]

    if labels_list:
        max_length = max([len(l) for l in labels_list])
    else:
        raise ValueError("The labels_list variable is empty.")

    labels_tensor = tf.ragged.constant(labels_list)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, boxes_list, labels_tensor))

    def load_and_preprocess(image_path, boxes, labels):
      image = tf.io.read_file(image_path)
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.resize(image, [512, 512])
      boxes = tf.RaggedTensor.from_row_lengths(boxes, [len(b) for b in boxes_list])
      labels = tf.RaggedTensor.from_row_lengths(labels, [len(l) for l in labels_list])
      return image, (boxes, labels)

    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [45]:
food_image_dir = os.path.join(food_images_path, train_subdir)
appliance_image_dir = os.path.join(appliance_images_path, train_subdir)

In [46]:
food_dataset = create_tf_dataset(food_annotations, food_image_dir)
appliance_dataset = create_tf_dataset(appliance_annotations, appliance_image_dir)

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [47]:
food_dataset = food_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
appliance_dataset = appliance_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

NameError: name 'food_dataset' is not defined

In [48]:
def conv_block(inputs, filters, kernel_size, strides=1):
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

In [31]:
def bifpn_block(inputs, num_channels):
    P3_in, P4_in, P5_in = inputs

    P3_up = layers.UpSampling2D()(P4_in)
    P3_out = layers.Concatenate()([P3_in, P3_up])
    P3_out = conv_block(P3_out, num_channels, 3)

    P4_up = layers.UpSampling2D()(P5_in)
    P4_out = layers.Concatenate()([P4_in, P4_up])
    P4_out = conv_block(P4_out, num_channels, 3)

    P4_down = layers.MaxPooling2D()(P3_out)
    P4_out = layers.Concatenate()([P4_out, P4_down])
    P4_out = conv_block(P4_out, num_channels, 3)

    P5_down = layers.MaxPooling2D()(P4_out)
    P5_out = layers.Concatenate()([P5_in, P5_down])
    P5_out = conv_block(P5_out, num_channels, 3)

    return P3_out, P4_out, P5_out

In [32]:
def build_efficientdet_cspdarknet(input_shape, num_classes_food, num_classes_appliance):
    inputs = layers.Input(shape=input_shape)

    backbone = tf.keras.applications.ResNet50(include_top=False, input_tensor=inputs, weights='imagenet')
    C3, C4, C5 = backbone.get_layer('conv3_block4_out').output, backbone.get_layer('conv4_block6_out').output, backbone.get_layer('conv5_block3_out').output

    num_channels = 64
    P3, P4, P5 = C3, C4, C5
    for _ in range(3):
        P3, P4, P5 = bifpn_block((P3, P4, P5), num_channels)

    food_head = layers.Conv2D(num_classes_food, 1, activation='sigmoid')(P3)
    food_head = layers.GlobalAveragePooling2D()(food_head)

    appliance_head = layers.Conv2D(num_classes_appliance, 1, activation='sigmoid')(P3)
    appliance_head = layers.GlobalAveragePooling2D()(appliance_head)

    model = models.Model(inputs, [food_head, appliance_head])
    return model

In [33]:
input_shape = (512, 512, 3)
num_classes_food = 10
num_classes_appliance = 5

model = build_efficientdet_cspdarknet(input_shape, num_classes_food, num_classes_appliance)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 512, 512, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 518, 518, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 256, 256, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 256, 256, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 256, 256, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 258, 258, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 128, 128, 64)   │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 128, 128, 64)   │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 128, 128, 64)   │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 128, 128, 64)   │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 128, 128, 64)   │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 128, 128, 64)   │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 128, 128, 64)   │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 128, 128, 256)  │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 128, 128, 256)  │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 27,830,607 (106.17 MB)

 Trainable params: 27,776,463 (105.96 MB)

 Non-trainable params: 54,144 (211.50 KB)

In [34]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

In [35]:
def train_step(images1, labels1, images2, labels2):
    with tf.GradientTape() as tape:
        preds1, preds2 = model(images1, training=True), model(images2, training=True)
        loss1 = loss_fn(labels1, preds1[0])
        loss2 = loss_fn(labels2, preds2[1])
        total_loss = loss1 + loss2
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return total_loss

In [36]:
def train_model(dataset1, dataset2, epochs=30):
    for epoch in range(epochs):
        for (images1, labels1), (images2, labels2) in zip(dataset1, dataset2):
            total_loss = train_step(images1, labels1, images2, labels2)
        print(f'Epoch {epoch + 1}, Loss: {total_loss.numpy()}')

In [37]:
train_model(food_dataset, appliance_dataset)

NameError: name 'food_dataset' is not defined

In [38]:
###single head model

In [39]:
def build_single_head_model(input_shape, num_classes, backbone_weights='imagenet'):
    inputs = layers.Input(shape=input_shape)

    backbone = tf.keras.applications.ResNet50(include_top=False, input_tensor=inputs, weights=backbone_weights)
    C3, C4, C5 = backbone.get_layer('conv3_block4_out').output, backbone.get_layer('conv4_block6_out').output, backbone.get_layer('conv5_block3_out').output

    num_channels = 64
    P3, P4, P5 = C3, C4, C5
    for _ in range(3):
        P3, P4, P5 = bifpn_block((P3, P4, P5), num_channels)

    head = layers.Conv2D(num_classes, 1, activation='sigmoid')(P3)
    head = layers.GlobalAveragePooling2D()(head)

    model = models.Model(inputs, head)
    return model

input_shape = (512, 512, 3)
num_classes_food = 10
num_classes_appliance = 5

food_model = build_single_head_model(input_shape, num_classes_food)
appliance_model = build_single_head_model(input_shape, num_classes_appliance)

food_model.summary()
appliance_model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 512, 512, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 518, 518, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 256, 256, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 256, 256, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 256, 256, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 258, 258, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 128, 128, 64)   │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 128, 128, 64)   │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 128, 128, 64)   │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 128, 128, 64)   │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 128, 128, 64)   │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 128, 128, 64)   │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 128, 128, 64)   │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 128, 128, 256)  │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 128, 128, 256)  │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 27,830,282 (106.16 MB)

 Trainable params: 27,776,138 (105.96 MB)

 Non-trainable params: 54,144 (211.50 KB)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 512, 512, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 518, 518, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 256, 256, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 256, 256, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 256, 256, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 258, 258, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 128, 128, 64)   │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 128, 128, 64)   │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 128, 128, 64)   │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 128, 128, 64)   │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 128, 128, 64)   │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 128, 128, 64)   │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 128, 128, 64)   │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 128, 128, 256)  │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 128, 128, 256)  │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 27,829,957 (106.16 MB)

 Trainable params: 27,775,813 (105.96 MB)

 Non-trainable params: 54,144 (211.50 KB)

In [40]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

def train_step_single_model(model, images, labels):
    with tf.GradientTape() as tape:
        preds = model(images, training=True)
        loss = loss_fn(labels, preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train_single_model(model, dataset, epochs=30):
    for epoch in range(epochs):
        for images, labels in dataset:
            loss = train_step_single_model(model, images, labels)
        print(f'Epoch {epoch + 1}, Loss: {loss.numpy()}')

train_single_model(food_model, food_dataset)
train_single_model(appliance_model, appliance_dataset)


NameError: name 'food_dataset' is not defined

In [49]:
food_test_dir = os.path.join(food_images_path, test_subdir)
appliance_test_dir = os.path.join(appliance_images_path, test_subdir)

food_test_annotations = load_annotations([os.path.join(food_annotations_path, f) for f in os.listdir(food_annotations_path) if f.startswith('test') and f.endswith('.json')])
appliance_test_annotations = load_annotations([os.path.join(appliance_annotations_path, f) for f in os.listdir(appliance_annotations_path) if f.startswith('test') and f.endswith('.json')])

food_test_dataset = create_tf_dataset(food_test_annotations, food_test_dir)
appliance_test_dataset = create_tf_dataset(appliance_test_annotations, appliance_test_dir)

food_test_dataset = food_test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
appliance_test_dataset = appliance_test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


ValueError: max() arg is an empty sequence

In [50]:
def evaluate_model(model, test_dataset):
    accuracy = tf.keras.metrics.CategoricalAccuracy()
    for images, labels in test_dataset:
        preds = model(images, training=False)
        accuracy.update_state(labels, preds)
    print(f'Test Accuracy: {accuracy.result().numpy()}')

print("Evaluating Food Model on Food Test Set")
evaluate_model(food_model, food_test_dataset)

print("Evaluating Appliance Model on Appliance Test Set")
evaluate_model(appliance_model, appliance_test_dataset)


Evaluating Food Model on Food Test Set


NameError: name 'food_test_dataset' is not defined

In [52]:
!pip install pylint